In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

root_dir = "../../"
sys.path.append(root_dir)

from utils import label_complexity_models as models, location_performance_t

data_dir = os.path.join(root_dir, "data")
rsna_dir = os.path.join(data_dir, "RSNA")
prediction_dir = os.path.join(rsna_dir, "predictions")
figure_dir = os.path.join(root_dir, "figures", "exam_level")
os.makedirs(figure_dir, exist_ok=True)

sns.set_theme()
sns.set_context("paper", font_scale=1.5)

In [ ]:
def exam_level_f1(cutoff):
    cutoff_method, cutoff_name = cutoff

    f1_df = []
    for model_title, model, weak_supervision in tqdm(models):
        if not os.path.exists(os.path.join(prediction_dir, model, "predictions_fixed")):
            continue

        m = int(model.split("/")[-2])
        r = int(model.split("/")[-1])

        prediction_df = pd.read_pickle(
            os.path.join(prediction_dir, model, "predictions_fixed")
        )
        prediction_df.set_index("series_idx", inplace=True)

        _df = location_performance_t(
            model_title,
            prediction_df,
            weak_supervision,
            cutoff=cutoff_method,
            offset=4,
            m=m,
        )

        _df = _df.groupby("est_name")["f1"].mean().reset_index()
        _df["m"] = m
        _df["r"] = r
        f1_df.append(_df)
    f1_df = pd.concat(f1_df)

    _, ax = plt.subplots(figsize=(16 / 2, 9 / 2))
    sns.lineplot(
        data=f1_df,
        x="m",
        y="f1",
        hue="est_name",
    )
    ax.set_xlabel(r"$m$")
    ax.set_ylabel(r"$f_1$ score")
    ax.set_xscale("log")
    ax.legend(title="", loc="lower right")
    ax.set_title(cutoff_name)

    plt.savefig(
        os.path.join(figure_dir, f"RSNA_f1_{cutoff_method}_label_complexity.jpg"),
        bbox_inches="tight",
    )
    plt.savefig(
        os.path.join(figure_dir, f"RSNA_f1_{cutoff_method}_label_complexity.pdf"),
        bbox_inches="tight",
    )
    plt.show()
    plt.close()

In [ ]:
cutoffs = [("youden", r"Youden's $J$ statistic"), ("d", r"Distance to $(0,1)$ point")]
for cutoff in cutoffs:
    exam_level_f1(cutoff)